In [1]:
!pip install transformers datasets scikit-learn torch

In [2]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import Dataset
import torch

In [3]:
# Load dataframe
df = pd.read_csv("prepended_v3_lemmatized.csv")
df.drop(columns=['text', 'text_length', 'length'], inplace=True)
df.rename(columns={'processed_text':'text', 'target':'label'}, inplace=True)
df = df[["text", "label"]].copy()  # Drop other columns if they exist
df["text"] = df["text"].astype(str)       # Force all entries to be strings
df

,text,label
0,ablaze wholesale market ablaze,1
1,ablaze always try bring heavy metal,0
2,ablaze africanbaze breaking newsnigeria flag s...,1
3,ablaze cry set ablaze,0
4,ablaze plus side look sky last night ablaze,0
...,...,...
7424,wrecked wrecked,0
7425,wrecked three day work theyve pretty wrecked h...,0
7426,wrecked fx forex trading cramer igers word wre...,0
7427,wrecked great atmosphere british lion gig toni...,0


In [4]:
dataset = Dataset.from_pandas(df)

In [5]:
print(dataset.column_names)

['text', 'label']


In [6]:
print(dataset[0])

{'text': 'ablaze wholesale market ablaze', 'label': 1}


In [7]:
print(type(dataset[0]["text"]))

<class 'str'>


In [8]:
# df.dtypes

In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

dataset = dataset.map(tokenize_function, batched=True)
dataset = dataset.rename_column("label", "labels")
dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/7429 [00:00<?, ? examples/s]

In [10]:
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

In [11]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Add this!
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-c84d39344df4>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cnsmith0611 (cnsmith0611-tennessee-technological-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

# Predict on new data
text = "The plot was really boring."
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
outputs = model(**inputs)
pred = torch.argmax(outputs.logits, dim=1).item()
print("Prediction:", pred)